# Fuel Moisture Field Observations from Hawaii

The purpose of this notebook is to clean and format data received from David Weise on his field study of fuel moisture content in Hawaii in 2000. Observations were taken of various plant species, live and dead, and weather data from portable station as well as a nearby RAWS.

## Background

- Published in 2004
- Used to compare Nelson model to other methods

**Outstanding Questions about Weise study:**
- How exactly was weather data in FM spreadsheet calculated from weather stations?
- Were Nelson model outputs compared to observed for each fuel species?
- For 3 samples were wet weight and dry weight switched?
    - 2242	OAHU	Makua	8/14/2000	700
    - 2275	OAHU	Makua	8/14/2000	2300
    - 2444	OAHU	Schofield	8/18/2000	700	

## Setup

In [1]:
import numpy as np
import pandas as pd
import os.path as osp

data_dir = "data/Hawaii_Weise_2000"

In [2]:
fm = pd.read_csv(osp.join(data_dir, "hawaiiFM_for_Hirschi.csv"))
sts = pd.read_csv(osp.join(data_dir, "site_location_for_Hirschi.csv"), skipinitialspace=True)
raws1999 = pd.read_csv(osp.join(data_dir, "PTA_1999_RAWS.csv"))
raws2000 = pd.read_csv(osp.join(data_dir, "HI_2000_RAWS.csv"))
raws2001 = pd.read_csv(osp.join(data_dir, "Portable_RAWS_2001.csv"))

## Clean

- Remove missing FMC observations
- Create date formatted column with time for FM and weather data
- Add spatial info if missing, fix site names
- Join weather data by year, add indicator if from portable station vs permanent RAWS

In [3]:
# Filter FM rows if missing fuel moisture, convert type
df = fm.copy()
df = df[~(df.Moisture_content.isna())]
df = df[~(df.Moisture_content == "*")]
df.Wet_weight = df.Wet_weight.astype(np.float64)
df.Dry_weight = df.Dry_weight.astype(np.float64)
df.Moisture_content = df.Moisture_content.astype(np.float64)

# Manually check FMC calculation
# NOTE: for a couple samples, wet weight is less than dry weight
fmc_manual = 100*((df.Wet_weight - df.Dry_weight)/df.Dry_weight)
np.max(np.abs(df.Moisture_content - fmc_manual))

29.64832357637041

In [4]:
# FM Data Time handling
df["Time"] = df["Time"].astype(str).str.zfill(4)  # ensures 4 digits
df["Time"] = df["Time"].str[:2] + ":" + df["Time"].str[2:]
df["date_time"] = pd.to_datetime(df["Date"] + " " + df["Time"])

# Print some checks
print(f"Earliest FM Observation: {df.date_time.min()}")
print(f"Latest FM Observation: {df.date_time.max()}")

Earliest FM Observation: 2000-08-11 00:00:00
Latest FM Observation: 2001-09-12 11:00:00


In [5]:
# FM Spatial Data Handling
print(f"{sts.site=}")
print()
print(f"{pd.Series(fm.Site.unique())=}")

# Rename stations spreadsheet to join (easier programmatically to change that one)
sts["site"] = sts['site'].replace({
    "Puʻu Waʻawaʻa": "Pu_u_Wa_awa_a",
    "Kaʻena_Pt": "Kaena_Point",
    "Polihale": "Polihale_Ridge",
    "PTA East": "PTAEast"
})
print()
print(f"Renaming stations dataframe")
print(f"All Station IDs in FM Dataframe: {np.all(sts.site.isin(df.Site))}")

# Add spatial columns to fm data
# Print checks, expect to see extra columns from sp data without changing rows
sp_cols = ["site", "lat", "long", "elev"]
print()
print(f"Merging Spatial Data")
print(f"Original {df.shape=}")
df = df.merge(sts[sp_cols], left_on="Site", right_on="site")
print(f"New {df.shape=}")

sts.site=0         Mauna_Kea
1     Puʻu Waʻawaʻa
2              HVNP
3              Kula
4          McGregor
5            Keanae
6         Kaʻena_Pt
7          Polihale
8             Makua
9         Schofield
10         PTA East
Name: site, dtype: object

pd.Series(fm.Site.unique())=0     Polihale_Ridge
1               HVNP
2           McGregor
3               Kula
4             Keanae
5        Kaena_Point
6      Pu_u_Wa_awa_a
7          Mauna_Kea
8              Makua
9          Schofield
10           PTAEast
dtype: object

Renaming stations dataframe
All Station IDs in FM Dataframe: True

Merging Spatial Data
Original df.shape=(2453, 14)
New df.shape=(2453, 18)


## Explore

In [6]:
# Pick one location and species
df2 = df[(df.Site == "Polihale_Ridge") & (df.Fuel_species == 'PITA&PIEL')]
print(f"Location Observation Times:")
print(f"Earliest: {df2.date_time.min()}")
print(f"Latest: {df2.date_time.max()}")

Location Observation Times:
Earliest: 2001-07-31 09:00:00
Latest: 2001-08-03 18:00:00


In [7]:
# 2 observations per time
df2.groupby(["date_time"]).agg("count")["Moisture_content"].unique()

array([2, 1])